In [1]:
import pandas as pd

In [3]:
!dir

 Volume in drive C is OSDisk
 Volume Serial Number is 4836-71B9

 Directory of C:\Users\rahulgup\Downloads\Kaggle_codes

04/30/2020  07:29 PM    <DIR>          .
04/30/2020  07:29 PM    <DIR>          ..
04/30/2020  07:29 PM    <DIR>          .ipynb_checkpoints
04/30/2020  03:18 PM           216,309 covid19-global-forecasting-week-2.zip
04/30/2020  07:28 PM           115,351 submission.csv
04/30/2020  07:28 PM           365,792 test.csv
04/30/2020  07:28 PM           783,368 train.csv
04/30/2020  07:29 PM                72 Untitled.ipynb
               5 File(s)      1,480,892 bytes
               3 Dir(s)  138,435,411,968 bytes free


In [8]:
train = pd.read_csv("train.csv", keep_default_na=False)

In [9]:
test = pd.read_csv("test.csv", keep_default_na=False)

In [10]:
submission = pd.read_csv("submission.csv", keep_default_na=False)

In [16]:
print(
    train.shape, test.shape, submission.shape
)

((20580, 6), (12642, 4), (12642, 3))


In [14]:
train.head(10)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,,Afghanistan,2020-01-22,0.0,0.0
1,2,,Afghanistan,2020-01-23,0.0,0.0
2,3,,Afghanistan,2020-01-24,0.0,0.0
3,4,,Afghanistan,2020-01-25,0.0,0.0
4,5,,Afghanistan,2020-01-26,0.0,0.0
5,6,,Afghanistan,2020-01-27,0.0,0.0
6,7,,Afghanistan,2020-01-28,0.0,0.0
7,8,,Afghanistan,2020-01-29,0.0,0.0
8,9,,Afghanistan,2020-01-30,0.0,0.0
9,10,,Afghanistan,2020-01-31,0.0,0.0


In [13]:
test.head(10)

,ForecastId,Province_State,Country_Region,Date
0,1,,Afghanistan,2020-03-19
1,2,,Afghanistan,2020-03-20
2,3,,Afghanistan,2020-03-21
3,4,,Afghanistan,2020-03-22
4,5,,Afghanistan,2020-03-23
5,6,,Afghanistan,2020-03-24
6,7,,Afghanistan,2020-03-25
7,8,,Afghanistan,2020-03-26
8,9,,Afghanistan,2020-03-27
9,10,,Afghanistan,2020-03-28


In [15]:
submission.head(10)

,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1
5,6,1,1
6,7,1,1
7,8,1,1
8,9,1,1
9,10,1,1


### K-Means clustering method considers two assumptions regarding the clusters – first that the clusters are spherical and second that the clusters are of similar size.
- Spherical assumption helps in separating the clusters when the algorithm works on the data and forms clusters. If this assumption is violated, the clusters formed may not be what one expects.
- On the other hand, assumption over the size of clusters helps in deciding the boundaries of the cluster. This assumption helps in calculating the number of data points each cluster should have.
- This assumption also gives an advantage. Clusters in K-means are defined by taking the mean of all the data points in the cluster. With this assumption, one can start with the centers of clusters anywhere.
- Keeping the starting points of the clusters anywhere will still make the algorithm converge with the same final clusters as keeping the centers as far apart as possible.

In [27]:
train.shape, len(train.Id.unique())

((20580, 6), 20580)

In [28]:
train.shape, len(train.Province_State.unique())

((20580, 6), 126)

In [29]:
train.shape, len(train.Country_Region.unique())

((20580, 6), 173)

In [30]:
train.shape, len(train.Date.unique())

((20580, 6), 70)

In [31]:
train.shape, len(train.ConfirmedCases.unique())

((20580, 6), 1536)

In [32]:
train.shape, len(train.Fatalities.unique())

((20580, 6), 328)

In [40]:
train.describe()

,Id,ConfirmedCases,Fatalities
count,20580.000000,20580.000000,20580.000000
mean,14685.500000,514.939116,21.080952
std,8487.230117,4541.261768,287.758197
min,1.000000,0.000000,0.000000
25%,7335.750000,0.000000,0.000000
50%,14685.500000,0.000000,0.000000
75%,22035.250000,35.000000,0.000000
max,29370.000000,105792.000000,12428.000000


In [63]:
df = train.groupby(train.Province_State).nunique()
df[df.Province_State > 1]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Province_State,,,,,,


In [60]:
df = train.groupby(train.Country_Region).nunique()
df[df.Province_State > 1]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Country_Region,,,,,,
Australia,560,8,1,70,112,9
Canada,700,10,1,70,120,20
China,2310,33,1,70,657,83
Denmark,210,3,1,70,51,17
France,700,10,1,70,87,27
Netherlands,280,4,1,70,47,24
US,3780,54,1,70,540,108
United Kingdom,490,7,1,70,67,23


In [61]:
df = train.groupby([train.Province_State, train.Country_Region]).nunique()
# df = train.groupby(train.Country_Region).nunique()
df[df.Province_State > 1]

,,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Province_State,Country_Region,,,,,,


In [54]:
train['Province_State'] = train.apply(
    lambda row: row['Country_Region'] if row['Province_State'] == "" else row['Province_State'],
    axis=1
)

In [55]:
train.groupby([train.Province_State, train.Country_Region]).nunique()

,,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Province_State,Country_Region,,,,,,
Afghanistan,Afghanistan,70,1,1,70,18,4
Alabama,US,70,1,1,70,20,5
Alaska,US,70,1,1,70,17,4
Albania,Albania,70,1,1,70,24,10
Alberta,Canada,70,1,1,70,23,5
Algeria,Algeria,70,1,1,70,28,18
Andorra,Andorra,70,1,1,70,17,6
Angola,Angola,70,1,1,70,7,2
Anhui,China,70,1,1,70,33,6


In [58]:
len(train.Province_State.unique()), len(train.Country_Region.unique())

(293, 173)